In [77]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import plotly.express as px
import glob
import spacy
from collections import defaultdict
import sys
sys.path.append('../../../../utils')
sys.path.append('..')
import gezi
from gezi import tqdm
from src.eval import calc_f1
from src.util import *
from src import config
from src.visualize import *
pd.set_option('display.float_format', lambda x: '%.02f' % x)

In [78]:
v = '10.0'
mn = 'pt.ro.xxx'
# mn2 = 'pt.ro.sent_loss.xxx'
mn2 = 'pt.ro.cls_sent.xxx'
classes = [
  'Claim', 'Evidence', 'Position', 'Concluding Statement', 'Lead', 'Counterclaim', 'Rebuttal'
]

In [79]:
model_dir = f'../working/offline/{v}/{mn}'
x = gezi.load(f'{model_dir}/valid.pkl')
model_dir2 = f'../working/offline/{v}/{mn2}'
x2 = gezi.load(f'{model_dir}/valid.pkl')

In [80]:
x.keys()

dict_keys(['id', 'label', 'mask', 'word_ids', 'sep', 'label_parts', 'label_type', 'label_mask', 'label_index', 'input_ids', 'attention_mask', 'pred', 'start_logits', 'parts', 'sent_logits'])

In [81]:
gezi.init_flags()
gezi.restore_configs(model_dir)

In [82]:
df_gt = pd.read_feather(f'{FLAGS.idir}/train.fea')
eval_ids = set(x['id'])
df_gt = df_gt[df_gt['id'].isin(eval_ids)]
df_gt = prepare(df_gt)
df_gt_ = df_gt.copy()
df_gt_['discourse_type'] = 'Claim'
df_gt.head()

  0%|          | 0/28809 [00:00<?, ?it/s]

  0%|          | 0/28808 [00:00<?, ?it/s]

,index,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring,text,essay_len,essay_words,start,end,discourse_words,gap_before,gap_length,gap_end_length,discourse_type_id
0,13790,00213CD75AC3,1622070442271.00,0,278,Cars have been an issue to our community for a...,Lead,Lead 1,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...,Cars have been an issue to our community for a...,2209,381,0,48,48,True,0.00,1931,5
1,13791,00213CD75AC3,1622070168679.00,306,354,limiting car usage could be an advantage to us.,Position,Position 1,54 55 56 57 58 59 60 61 62,Cars have been an issue to our community for a...,2209,381,54,63,9,True,28.00,1855,3
2,13792,00213CD75AC3,1622070209988.00,354,693,"In Bogata, Columbia, barely any Columbians use...",Evidence,Evidence 1,63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 7...,Cars have been an issue to our community for a...,2209,381,63,117,54,False,nan,1516,2
3,13793,00213CD75AC3,1622070200612.00,693,779,limiting car usage would be less stressful by ...,Claim,Claim 1,117 118 119 120 121 122 123 124 125 126 127 12...,Cars have been an issue to our community for a...,2209,381,117,133,16,False,nan,1430,1
4,13794,00213CD75AC3,1622070238370.00,779,935,People have been fined for causing accidents d...,Evidence,Evidence 2,133 134 135 136 137 138 139 140 141 142 143 14...,Cars have been an issue to our community for a...,2209,381,133,161,28,False,nan,1274,2


In [83]:
model = get_model()
gezi.load_weights(model, model_dir)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
ic| util.py:2767 in load_weights()
    model_path: '../working/offline/10.0/pt.ro.xxx/model.pt'


In [84]:
model2 = get_model()
gezi.load_weights(model2, model_dir2)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
ic| util.py:2767 in load_weights()
    model_path: '../working/offline/10.0/pt.ro.cls_sent.xxx/model.pt'


In [85]:
x['start_logits'].shape

(3119, 512, 2)

In [86]:
seps = ((x['start_logits'][:,:,1] - x['start_logits'][:,:,0]) > 0).astype(int)

In [87]:
def sep2index(seps, mask):
  label_index = np.zeros_like(seps)
  for i, sep in enumerate(seps):
    idx = 0
    for j in range(len(sep)):
      if sep[j] == 1:
        idx += 1
      label_index[i, j] = idx
  return np.asarray(label_index * mask)

In [88]:
label_index = sep2index(seps, x['mask'])

In [89]:
label_index

array([[ 0,  1,  1, ..., 10, 10,  0],
       [ 0,  1,  1, ...,  0,  0,  0],
       [ 0,  1,  1, ..., 12, 12,  0],
       ...,
       [ 0,  1,  1, ..., 16, 16,  0],
       [ 0,  1,  1, ..., 14, 14,  0],
       [ 0,  1,  1, ...,  0,  0,  0]])

In [90]:
inputs = {
  'input_ids': x['input_ids'],
  'attention_mask': x['attention_mask'],
  'mask': x['mask'],
  'label_index': label_index
  # 'label_index': x['label_index']
}

In [91]:
x.keys()

dict_keys(['id', 'label', 'mask', 'word_ids', 'sep', 'label_parts', 'label_type', 'label_mask', 'label_index', 'input_ids', 'attention_mask', 'pred', 'start_logits', 'parts', 'sent_logits'])

In [92]:
inputs['id'] = np.zeros_like(inputs['mask'][:,0])

In [93]:
# using new, model2 to predict 
p = gezi.predict(model2, inputs, 32)

Predicting:   0%|          | 0/98 [00:00<?, ?it/s]

In [94]:
p.keys()

dict_keys(['pred', 'start_logits', 'parts', 'sent_logits'])

In [95]:
p['sent_logits'].shape

(3119, 40, 8)

In [96]:
types = np.argmax(p['sent_logits'], -1)

In [97]:
parts = seps.sum(-1)

In [98]:
parts

array([10, 10, 12, ..., 16, 14,  7])

In [99]:
types

array([[5, 1, 1, ..., 0, 0, 0],
       [0, 3, 2, ..., 0, 0, 0],
       [0, 5, 3, ..., 0, 0, 0],
       ...,
       [5, 3, 0, ..., 0, 0, 0],
       [0, 5, 3, ..., 0, 0, 0],
       [3, 1, 2, ..., 0, 0, 0]])

In [100]:
ids = x['id']
ids

array(['367111CA290A', 'C0EF97669300', '0F1B167D414E', ...,
       '0D6F5D6E2E77', 'E83BEEE4EF18', 'DBFD15D537CA'], dtype='<U12')

In [101]:
len(ids)

3119

In [102]:
word_ids_list = x['word_ids']

In [103]:
preds_list = []
types_list = []
ids_list = []
for i, id in tqdm(enumerate(ids), total=len(ids)):
  type_list = []
  pred_list = []
  num_parts = parts[i]
  word_ids = word_ids_list[i]
  word_id_ = None
  preds = []
  for j, word_id in enumerate(word_ids):
    if word_id < 0:
      continue
    if word_id == word_id_:
      continue
    else:
      word_id_ = word_id
    if seps[i, j] == 1:
      if len(preds) >= 2 and pre_type != 0:
        pred_list.append(' '.join(preds))
        type_list.append(id2dis[pre_type])
        # print(pred_list[-1], type_list[-1])
      preds = []
    pre_type = types[i, label_index[i, j] - 1]
    preds.append(str(word_id))
  if len(preds) >= 2 and pre_type != 0:
    pred_list.append(' '.join(preds))
    type_list.append(id2dis[pre_type])
  ids_list.extend([id] * len(type_list))
  preds_list.extend(pred_list)
  types_list.extend(type_list)


  0%|          | 0/3119 [00:00<?, ?it/s]

In [104]:
len(preds_list)

27050

In [105]:
len(types_list)

27050

In [106]:
len(ids_list)

27050

In [107]:
  m = {
    'id': ids_list,
    'class': types_list,
    'predictionstring': preds_list
  }

  df_pred = pd.DataFrame(m)

In [108]:
df_pred

,id,class,predictionstring
0,367111CA290A,Lead,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
1,367111CA290A,Claim,60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 7...
2,367111CA290A,Claim,93 94 95 96 97 98 99 100 101 102
3,367111CA290A,Evidence,103 104 105 106 107 108 109 110 111 112 113 11...
4,367111CA290A,Evidence,186 187 188 189 190 191 192 193 194 195 196 19...
...,...,...,...
27045,DBFD15D537CA,Evidence,40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 5...
27046,DBFD15D537CA,Evidence,80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 9...
27047,DBFD15D537CA,Claim,179 180 181 182 183 184 185 186 187 188 189 19...
27048,DBFD15D537CA,Evidence,193 194 195 196 197 198 199 200 201 202 203 20...


In [109]:
x

{'id': array(['367111CA290A', 'C0EF97669300', '0F1B167D414E', ...,
        '0D6F5D6E2E77', 'E83BEEE4EF18', 'DBFD15D537CA'], dtype='<U12'),
 'label': array([[0, 0, 0, ..., 2, 2, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 2, 2, 0],
        ...,
        [0, 5, 5, ..., 4, 4, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 3, 3, ..., 0, 0, 0]], dtype=int32),
 'mask': array([[0, 1, 1, ..., 1, 1, 0],
        [0, 1, 1, ..., 0, 0, 0],
        [0, 1, 1, ..., 1, 1, 0],
        ...,
        [0, 1, 1, ..., 1, 1, 0],
        [0, 1, 1, ..., 1, 1, 0],
        [0, 1, 1, ..., 0, 0, 0]], dtype=int32),
 'word_ids': array([[ -1,   0,   1, ..., 465, 466,  -1],
        [ -1,   0,   0, ...,  -1,  -1,  -1],
        [ -1,   0,   0, ..., 450, 451,  -1],
        ...,
        [ -1,   0,   1, ..., 457, 458,  -1],
        [ -1,   0,   0, ..., 467, 468,  -1],
        [ -1,   0,   0, ...,  -1,  -1,  -1]], dtype=int32),
 'sep': array([[0, 1, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0],
        [

In [110]:
list(x.keys())

['id',
 'label',
 'mask',
 'word_ids',
 'sep',
 'label_parts',
 'label_type',
 'label_mask',
 'label_index',
 'input_ids',
 'attention_mask',
 'pred',
 'start_logits',
 'parts',
 'sent_logits']

In [111]:
#----------- 句子分类准确率, f1
from sklearn.metrics import accuracy_score, f1_score
sent_logits = x['sent_logits']
sent_pred = np.argmax(p['sent_logits'], axis=-1)
y, y_, w = x['label_type'].reshape(-1), sent_pred.reshape(-1), x['label_mask'].reshape(-1)
acc = accuracy_score(y, y_, sample_weight=w)
f1 = f1_score(y, y_, average='macro', sample_weight=w)
acc, f1

(0.47127744633741314, 0.43064537878765435)

In [112]:
df_pred = pd.merge(df_pred, df_gt[['id', 'text']].drop_duplicates(), on=['id'], how='left')
df_pred = prepare(df_pred)
df_pred_ = df_pred.copy()
df_pred_['discourse_type'] = 'Claim'
df_pred.head()

  0%|          | 0/27050 [00:00<?, ?it/s]

  0%|          | 0/27049 [00:00<?, ?it/s]

,id,discourse_type,predictionstring,text,start,essay_len,essay_words,discourse_start,discourse_end,gap_before,gap_length,gap_end_length
0,00213CD75AC3,Lead,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...,Cars have been an issue to our community for a...,0,2209,381,0,123,True,0.00,2086
1,00213CD75AC3,Lead,24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 3...,Cars have been an issue to our community for a...,24,2209,381,123,278,False,nan,1931
2,00213CD75AC3,Position,48 49 50 51 52 53 54 55 56 57 58 59 60 61 62,Cars have been an issue to our community for a...,48,2209,381,278,354,False,nan,1855
3,00213CD75AC3,Evidence,63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 7...,Cars have been an issue to our community for a...,63,2209,381,354,596,False,nan,1613
4,00213CD75AC3,Evidence,102 103 104 105 106 107 108 109 110 111 112 11...,Cars have been an issue to our community for a...,102,2209,381,596,693,False,nan,1516


In [113]:
res = {}
f1s = []
CLASSES = list(id2dis.values())[1:]
for c in CLASSES:
  ret = calc_f1(df_gt[df_gt['discourse_type'] == c],
               df_pred[df_pred['discourse_type'] == c], return_dict=True)
  res[c] = ret['f1']
  res[f'{c}/acc'] = ret['acc']
  res[f'{c}/recall'] = ret['recall']
  f1s.append(ret['f1'])
res['Overall'] = np.mean(f1s)
res

{'Claim': 0.6003748208576783,
 'Claim/acc': 0.6746778989098117,
 'Claim/recall': 0.5408142999006952,
 'Evidence': 0.618530416602927,
 'Evidence/acc': 0.5822213689161947,
 'Evidence/recall': 0.6596693495758104,
 'Position': 0.6650358773646445,
 'Position/acc': 0.6685245901639344,
 'Position/recall': 0.6615833874107723,
 'Concluding Statement': 0.5818181818181818,
 'Concluding Statement/acc': 0.6420198376916141,
 'Concluding Statement/recall': 0.5319387373926037,
 'Lead': 0.7600997506234414,
 'Lead/acc': 0.710820895522388,
 'Lead/recall': 0.8167202572347267,
 'Counterclaim': 0.4175704989154013,
 'Counterclaim/acc': 0.5347222222222222,
 'Counterclaim/recall': 0.34252669039145905,
 'Rebuttal': 0.33199033037872683,
 'Rebuttal/acc': 0.4703196347031963,
 'Rebuttal/recall': 0.25653798256537985,
 'Overall': 0.5679171252230002}

In [114]:
stop

NameError: name 'stop' is not defined

In [ ]:
!ls ../working/offline/7.0/pt.large.x1

In [ ]:
eval_ids = list(eval_ids)
f1s = []
for id in tqdm(eval_ids):
  f1s.append(calc_f1(df_gt[df_gt['id'] == id], df_pred[df_pred['id'] == id]))

In [ ]:
df = pd.DataFrame(
  {
    'id': eval_ids,
    'f1': f1s
  }
)
df

In [ ]:
df.describe()

In [ ]:
df.sort_values(['f1'], ascending=True, inplace=True)

In [ ]:
df.id.values

In [ ]:
def visualize(essay):
  f1 = calc_f1(
      df_gt[df_gt['id'] == essay],
      df_pred[df_pred['id'] == essay].rename({'discourse_type': 'class'},
                                             axis=1))
  f1_binary = calc_f1(
      df_gt_[df_gt_['id'] == essay],
      df_pred_[df_pred_['id'] == essay].rename({'discourse_type': 'class'},
                                               axis=1))
  print(essay, 'f1:', f1, 'f1_binary:', f1_binary)
  visulize_df(df_gt, essay)
  visulize_df(df_pred, essay)

In [ ]:
visualize(df.id.values[0])

In [ ]:
visualize(df.id.values[1])

In [ ]:
visualize(df.id.values[2])

In [ ]:
visualize(df.id.values[3])

In [ ]:
visualize(df.id.values[5])

In [ ]:
visualize(df.id.values[random.randint(0, len(eval_ids) - 1)])

In [ ]:
visualize(df.id.values[-1])

In [ ]:
visualize('193591CD2E86')